In [1]:
import sys
sys.path.append('../')

from model import *
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import pickle as pkl
import matplotlib.pyplot as plt
import sklearn

### Data

In [2]:
with open('../../data/ChildWelfare/X_preprocess.pkl', 'rb') as handle:
    X,screener_ids,refer_ids,Y_obs,D,Y_serv,Y_sub,colnames = pkl.load(handle)

In [3]:
#drop instances if expert assessed a single case

drop_experts = []
for num in screener_ids:

    if screener_ids.count(num) < 10:

        drop_experts.append(num)

In [4]:
drop_idx = []
for index, elem in enumerate(screener_ids):
    if elem in drop_experts:
        drop_idx.append(index)

In [5]:
X = np.delete(X,drop_idx,axis=0)
Y_serv = np.delete(Y_serv,drop_idx,axis=0)
Y_sub = np.delete(Y_sub,drop_idx,axis=0)
Y_obs = np.delete(Y_obs,drop_idx,axis=0)
D = np.delete(D,drop_idx,axis=0)
refer_ids = np.delete(refer_ids,drop_idx,axis=0)
screener_ids = np.delete(screener_ids,drop_idx,axis=0)

In [6]:
X.shape

(46528, 216)

In [7]:
D = D.reshape((D.shape[0],))
Y_obs = Y_obs.reshape((Y_obs.shape[0],))

In [8]:
target = pd.DataFrame({'D': D, 'Y1': Y_obs, 'Y2': Y_serv, 'Y3': Y_sub})

Split data in a 80% train, 20% test

In [9]:
cov_train, cov_test, tar_train, tar_test, nur_train, nur_test = train_test_split(pd.DataFrame(X), target, pd.Series(screener_ids), test_size = 0.2, random_state = 0)

### Modelling

In [10]:
# Model's characteristics
#[(100,), (200,), (100,50), (200,50),(200,100),(200,100,50)]
params = {'layers': [200, 100]} # If = [] equivalent to a simple logistic regression

# Amalgation parameters
rho = 0.02 # Control which point to consider from a confience point of view
pi_1 = 4.0 # Control criterion on centre mass metric
pi_2 = 0.8 # Control criterion on opposing metric
tau = 1.0  # Balance between observed and expert labels

##### 1. Train on decision

This model models the nurse decision based on covariates

In [11]:
for l1_penalty in [ 0.001, 0.01, 0.1]:
    try:
        model = BinaryMLP(**params)
        model = model.fit(cov_train, tar_train['D'], nur_train, l1_penalty = l1_penalty)
        break
    except Exception as e:
        print(e, l1_penalty)
        pass

Loss: 0.428:  14%|█▎        | 137/1000 [08:20<49:49,  3.46s/it]  

In [12]:
Y_pred_h_test = model.predict(cov_test)

In [13]:
# Naive performance
roc_auc_score(tar_test['Y1'], model.predict(cov_test))

0.6755354470952571

In [14]:
# Yc performance
roc_auc_score(tar_test['Y2'], model.predict(cov_test))

0.8617985755262902

In [15]:
roc_auc_score(tar_test['Y3'], model.predict(cov_test))

0.9722197481441487

In [16]:
roc_auc_score(tar_test['D'], model.predict(cov_test))

0.8974737921173025

##### 2. Agreement computation 

Measure of agreeability are estimated in a cross validation fashion on the train set.

In [17]:
# Fold evaluation of influences
folds, predictions, influence = influence_cv(BinaryMLP, cov_train, tar_train['D'], nur_train, params = params, fit_params = {"l1_penalty": l1_penalty})


Loss: 0.465:   6%|▌         | 61/1000 [02:08<33:43,  2.15s/it]


Loss: 0.439:   9%|▊         | 87/1000 [03:21<39:10,  2.57s/it]

ValueError: Architecture leads to singular weights matrix for last layer: Use another architecture or increase l1_penalty.

In [ ]:
# Compute metrics agreeability
center_metric, opposing_metric = compute_agreeability(influence)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(center_metric, opposing_metric)

In [ ]:
# Apply criteria on amalgamation
high_conf = (predictions > (1 - rho))# | (predictions < rho)
high_agr = (center_metric > pi_1) & (opposing_metric > pi_2) & high_conf
high_agr_correct = ((predictions - tar_train['D']).abs() < rho) & high_agr

In [ ]:
plt.hist(predictions)

In [ ]:
print('high_conf:', sum(high_conf))
print('high_agr:', sum(high_agr))
print('high_agr_correct:', sum(high_agr_correct))

In [ ]:
from sklearn.calibration import calibration_curve
prob_true, prob_pred = calibration_curve(tar_train['D'], predictions, n_bins=7)
plt.plot(prob_true,prob_pred, marker='o', linewidth=1, label='logreg')


In [ ]:
# Create amalgamated labels
tar_train['Ya'] = tar_train['Y1'].copy()
tar_train['Ya'][high_agr_correct] = (1 - tau) * tar_train['Y1'][high_agr_correct] \
                                    + tau * tar_train['D'][high_agr_correct]

In [ ]:
sum(tar_train['D']!=tar_train['Y1'])

In [ ]:
sum(tar_train['Ya']!=tar_train['Y1'])

In [ ]:
#index for selective labels
index_amalg = [i==1.0 for i in tar_train['D']] | high_agr_correct

##### 3. Updated model

In [ ]:
#model with selective labels
model = BinaryMLP(**params)
model = model.fit(cov_train[index_amalg], tar_train[index_amalg]['Ya'], nur_train[index_amalg])

In [ ]:
Y_pred_amalg_test = model.predict(cov_test)

In [ ]:
# #model without selective labels
# model = BinaryMLP(**params)
# model = model.fit(cov_train, tar_train['Ya'], nur_train[index_amalg])

In [ ]:
# Naive performance
roc_auc_score(tar_test['Y1'], model.predict(cov_test))

In [ ]:
# Yc performance
roc_auc_score(tar_test['Y2'],model.predict(cov_test))

In [ ]:
roc_auc_score(tar_test['Y3'],model.predict(cov_test))

In [ ]:
roc_auc_score(tar_test['D'],model.predict(cov_test))

##### 4. Train on observed data

In [ ]:
model = BinaryMLP(**params)
model = model.fit(cov_train[tar_train['D']==1], tar_train['Y1'][tar_train['D']==1], nur_train)

In [ ]:
Y_pred_obs_test = model.predict(cov_test)

In [ ]:
# Naive performance
roc_auc_score(tar_test['Y1'], model.predict(cov_test))

In [ ]:
# Yc performance
roc_auc_score(tar_test['Y2'],model.predict(cov_test))

In [ ]:
roc_auc_score(tar_test['Y3'],model.predict(cov_test))

In [ ]:
roc_auc_score(tar_test['D'],model.predict(cov_test))

### Human

In [ ]:
#if models trained without 1 penalty, retrain f_h
model = BinaryMLP(**params)
model = model.fit(cov_train, tar_train['D'], nur_train)

In [ ]:
Y_pred_h_test = model.predict(cov_test)

print(roc_auc_score(tar_test['Y1'], model.predict(cov_test)))

print(roc_auc_score(tar_test['Y2'], model.predict(cov_test)))

print(roc_auc_score(tar_test['Y3'], model.predict(cov_test)))

print(roc_auc_score(tar_test['D'], model.predict(cov_test)))

#### Hybrid

In [ ]:
high_agr.shape

In [ ]:
# Y_pred_hybrid_test = np.copy(Y_pred_obs_test)
# Y_pred_hybrid_test[high_agr_test] = Y_pred_h_test[high_agr_test]

## Precision/Accuracy

In [ ]:
def eval_pred_outcome(Y_outcome_test, Y_pred_test, Y_h_test, p):
    Y_pred_test_scrin = Y_pred_test[Y_h_test==1]
    Y_outcome_test_scrin = Y_outcome_test[Y_h_test==1]
    #print(sum(Y_outcome_test_scrin))
    idx_25qrtl_pred = np.argsort(Y_pred_test_scrin)[::-1][:int(np.floor(p*len(Y_pred_test_scrin)))]
    #baseline
    baseline = sum(Y_outcome_test_scrin)/len(Y_outcome_test_scrin)
    #print(idx_25qrtl_pred )
    #print(sum(Y_outcome_test_scrin[idx_25qrtl_pred]))
    #precision and recall in top p% highsest scored screen-in
    precision_p = sum(Y_outcome_test_scrin[idx_25qrtl_pred])/len(idx_25qrtl_pred)
    recall_p = sum(Y_outcome_test_scrin[idx_25qrtl_pred])/sum(Y_outcome_test)
    
    return baseline, precision_p, recall_p

In [ ]:
print('Eval resp. Y_obs')
print('f_y', eval_pred_outcome(np.array(tar_test['Y1']).flatten(), Y_pred_obs_test.flatten(), np.array(tar_test['D']==1).flatten(), 0.25))
print('f_A', eval_pred_outcome(np.array(tar_test['Y1']).flatten(), Y_pred_amalg_test.flatten(), np.array(tar_test['D']==1).flatten(), 0.25))
print('f_h', eval_pred_outcome(np.array(tar_test['Y1']).flatten(), Y_pred_h_test.flatten(), np.array(tar_test['D']==1).flatten(), 0.25))

In [ ]:
print('Eval resp. Y_serv')
print('f_y', eval_pred_outcome(np.array(tar_test['Y2']).flatten(), Y_pred_obs_test.flatten(), np.array(tar_test['D']==1).flatten(), 0.25))
print('f_A', eval_pred_outcome(np.array(tar_test['Y2']).flatten(), Y_pred_amalg_test.flatten(), np.array(tar_test['D']==1).flatten(), 0.25))
print('f_h', eval_pred_outcome(np.array(tar_test['Y2']).flatten(), Y_pred_h_test.flatten(), np.array(tar_test['D']==1).flatten(), 0.25))

In [ ]:
print('Eval resp. Y_sub')
print('f_y', eval_pred_outcome(np.array(tar_test['Y3']).flatten(), Y_pred_obs_test.flatten(), np.array(tar_test['D']==1).flatten(), 0.25))
print('f_A', eval_pred_outcome(np.array(tar_test['Y3']).flatten(), Y_pred_amalg_test.flatten(), np.array(tar_test['D']==1).flatten(), 0.25))
print('f_h', eval_pred_outcome(np.array(tar_test['Y3']).flatten(), Y_pred_h_test.flatten(), np.array(tar_test['D']==1).flatten(), 0.25))

In [ ]:
def eval_pred(Y_pred_test, tar_test, p):
    dic_k_Y_pred = {}
    
    base_ooh, precision_ooh, recall_ooh = eval_pred_outcome(np.array(tar_test['Y1']).flatten(), Y_pred_test.flatten(), np.array(tar_test['D']==1).flatten(), p)
    base_serv, precision_serv, recall_serv = eval_pred_outcome(np.array(tar_test['Y2']).flatten(), Y_pred_test.flatten(), np.array(tar_test['D']==1).flatten(), p)
    base_sub, precision_sub, recall_sub = eval_pred_outcome(np.array(tar_test['Y3']).flatten(), Y_pred_test.flatten(), np.array(tar_test['D']==1).flatten(), p)
    
    dic_k_Y_pred['ooh'] = {'base': float(base_ooh), 'precision': float(precision_ooh), 'recall': float(recall_ooh)}
    dic_k_Y_pred['sub'] = {'base': float(base_sub), 'precision': float(precision_sub), 'recall': float(recall_sub)}
    dic_k_Y_pred['serv'] = {'base': float(base_serv), 'precision': float(precision_serv), 'recall': float(recall_serv)}
    
    return dic_k_Y_pred
    

In [ ]:
def eval_k_iter(Y_pred_obs_test, Y_pred_amalg_test, Y_pred_h_test, tar_test, p):
    dic_eval = {}
    
    dic_k = {}
    dic_k['Y_pred_obs_test'] = eval_pred(Y_pred_obs_test, tar_test, p)
    dic_k['Y_pred_h_test'] = eval_pred(Y_pred_h_test, tar_test, p)
    #dic_k['Y_pred_hybrid_test'] = eval_pred(Y_pred_hybrid_test, Y_h, Y_ooh, Y_serv, Y_sub, refer_ids_test, refer_ids, p)
    #print('done with Y_pred_hyb_test')
    dic_k['Y_pred_amalg_test'] = eval_pred(Y_pred_amalg_test, tar_test, p)

    
    dic_eval[0] = dic_k
    
    return dic_eval

In [ ]:
list_dics =[]
for p in np.arange(0.05, 0.4, 0.05):
    list_dics.append(eval_k_iter(Y_pred_obs_test, Y_pred_amalg_test, Y_pred_h_test, tar_test, p))

In [ ]:
dic_25 = eval_k_iter(Y_pred_obs_test, Y_pred_amalg_test, Y_pred_h_test, tar_test, 0.25)


In [ ]:
def plot_perf_bar(dic_eval, models = ['Y_pred_obs_test', 'Y_pred_h_test',
                                     'Y_pred_hybrid_test', 'Y_pred_amalg_test'], 
                  plot_labels = [ '$f_y$', '$f_h$', '$f_\mathcal{hyb}$',  '$f_\mathcal{A}$' ] ,
                  labels = ['ooh', 'sub', 'serv'], y_max=1.05, legend_appear=True,
                  metric ='precision', fig_title='fig_noname'):
    
    K = len(dic_eval.keys())
    
    n_labels = len(labels)
    
    fig, ax = plt.subplots(figsize=(10,5))
    index = np.arange(n_labels)
    bar_width = 0.15
    opacity = 0.8
    colors = ['green', 'darkred', 'yellow', 'blue']
    hatches = ['//', '-', '\\', '.']
    
    
    m_idx =0
    for m in models:
        plot_mean = [0]*n_labels
        plot_std = [0]*n_labels
        
        l_idx = 0
        for l in labels:
            metric_list = [0]*K
            if metric == 'precision':
                for k in range(K):
                    metric_list[k] = dic_eval[k][m][l]['precision']
            
            if metric == 'recall':
                for k in range(K):
                    metric_list[k] = dic_eval[k][m][l]['recall']
            plot_mean[l_idx] = np.mean(metric_list)  
            plot_std[l_idx] = np.std(metric_list)  
            l_idx += 1
        rects = plt.bar(index+m_idx*bar_width, plot_mean, yerr= plot_std,
            alpha=opacity, color=colors[m_idx], capsize=7,  width=.1, hatch = hatches[m_idx],
                            label=plot_labels[m_idx])        
            
            
        m_idx+=1                
    
    
    for n in index[:-1]:
        if metric=='precision':
            t= 0.8
        else:
            t=0.7
        plt.axvline(x=n+t, color='grey', linestyle='--')
        
    if metric=='precision':
        plot_mean = [0]*n_labels
        plot_std = [0]*n_labels
        l_idx = 0
        for l in labels:
            metric_list = [0]*K
            for k in range(K):
                metric_list[k] = dic_eval[k][m][l]['base']
            plot_mean[l_idx] = np.mean(metric_list)  
            plot_std[l_idx] = np.std(metric_list) 
            l_idx +=1
        rects = plt.bar(index+len(models)*bar_width, plot_mean, yerr= plot_std,
            alpha=opacity, color='grey',capsize=7,  width=.1,label='Overall prev.')     

    plt.ylabel(metric,fontsize=25)
    #plt.ylim((0,1.05))
    #plt.title('Performance of outcomes by model',fontsize=18)
    plt.xticks(index + 1.5*bar_width, ('OOH', 'Substantiated', 'Services'),fontsize=18)
    plt.yticks(fontsize=16)
    if legend_appear==True:
        plt.legend(loc = 'upper left', fontsize=18)
    plt.ylim(0.0,y_max)
    plt.tight_layout()

#     plt.savefig(fig_title, dpi=300,
#             orientation='portrait' )
    plt.show()

In [ ]:
plot_perf_bar(dic_25, models = ['Y_pred_obs_test', 'Y_pred_h_test', 'Y_pred_amalg_test'], 
                  plot_labels = [ '$f_y$', '$f_h$',   '$f_\mathcal{A}$' ])

In [ ]:
def plot_perf_bar(list_dics, index = np.arange(0.05, 0.4, 0.05), models = ['Y_pred_obs_test', 'Y_pred_h_test',
                                     'Y_pred_hybrid_test', 'Y_pred_amalg_test'], 
                  plot_labels = [ '$f_y$', '$f_h$', '$f_\mathcal{hyb}$',  '$f_\mathcal{A}$' ] ,
                  labels = ['ooh', 'sub', 'serv'], y_max=1.05, legend_appear=True,
                  metric ='precision', fig_title='fig_noname'):
    
    K = len(list_dics[0].keys())
    
    n_groups = len(labels)
    
    fig, ax_list = plt.subplots(1,n_groups, figsize=(14,5))
    N=len(list_dics)
    
    bar_width = 0.1
    opacity = 0.8
    colors = ['green', 'darkred', 'goldenrod', 'blue']
    hatches = ['//', '-', '\\', '.']
    l_idx = 0
    for l in labels:
        m_idx =0
        for m in models:
            plot_mean = [0]*N
            plot_std = [0]*N

            dic_idx = 0
            for dic_eval in list_dics:
                metric_list = [0]*K
                if metric == 'precision':
                    for k in range(K):
                        metric_list[k] = dic_eval[k][m][l]['precision']

                if metric == 'recall':
                    for k in range(K):
                        metric_list[k] = dic_eval[k][m][l]['recall']
                plot_mean[dic_idx] = np.mean(metric_list)  
                plot_std[dic_idx] = np.std(metric_list)  
                dic_idx += 1
            ax_list[l_idx].errorbar(index, plot_mean, yerr= plot_std,
                alpha=opacity, color=colors[m_idx], capsize=7, 
                               label=plot_labels[m_idx])        
#             else:
#                 ax_list[l_idx].errorbar(index, plot_mean, yerr= plot_std,
#                 alpha=opacity, color=colors[m_idx], capsize=7)     

            m_idx+=1                
        

#         for n in index[:-1]:
#             if metric=='precision':
#                 t= (len(list_dics)-1)*bar_width+0.2
#             else:
#                 t= len(list_dics)*bar_width
#             plt.axvline(x=n+t, color='grey', linestyle='--')

        if metric=='precision':
            plot_mean = [0]*N
            plot_std = [0]*N
            dic_idx = 0
            for dic_eval in list_dics:
                metric_list = [0]*K
                for k in range(K):
                    metric_list[k] = dic_eval[k][m][l]['base']
                plot_mean[dic_idx] = np.mean(metric_list)  
                plot_std[dic_idx] = np.std(metric_list) 
                dic_idx +=1
            ax_list[l_idx].errorbar(index, plot_mean, yerr= plot_std,
                alpha=opacity, color='grey',capsize=7, label='base')     
#             else:
#                 ax_list[l_idx].errorbar(index, plot_mean, yerr= plot_std,
#                 alpha=opacity, color='grey',capsize=7)     
        l_idx+=1
        
        
   # plt.ylabel(metric,fontsize=25)
    ax_list[0].set_ylabel(metric,fontsize=20)
    ax_list[1].set_xlabel('top p%',fontsize=20)
    
    for k in range(n_groups):
        ax_list[k].set_title(labels[k],fontsize=20)
    
    
    ax_list[2].legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize=18)
    
    #ax_list[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
        #  fancybox=True, shadow=True, ncol=5)
    
    #plt.xticks(index+dic_idx*bar_width)
    
#     plt.yticks(fontsize=16)
#     if legend_appear==True:
#         plt.legend(loc = 'upper left', fontsize=18)
    #plt.ylim(0.0,y_max)
    #plt.tight_layout()

#     plt.savefig(fig_title, dpi=300,
#             orientation='portrait' )
    plt.show()

In [ ]:
plot_perf_bar(list_dics, index = np.arange(0.05, 0.4, 0.05), models = ['Y_pred_obs_test', 'Y_pred_h_test','Y_pred_amalg_test'], 
                  plot_labels = [ '$f_y$', '$f_h$',  '$f_\mathcal{A}$' ])